# Tensorboard with decomon


In this notebook, we show how to have a look to the graph of a decomon model.

We use here the same model as in [tutorial 1](../tutorial1_sinus-interactive.ipynb) and you should refer to it for any details about how it works.


## Prerequisites

Because decomon models have specificities, visualizing them with tensorboard reveal some bug in the library and we need to get the last version of tensorboard available on master to make the notebook work. (The bug fix should be included in future 2.13.0)




### On local Jupyter

Here is a way to install decomon with the nightly version of tensorflow (including the nighly version of tensorboard) available as package "tf-nightly". We need to install the dependencies of decomon manually and then to use --no-deps option to install decomon, because tf-nightly is not recognized as tensorflow by pip.


```shell
# create and activate a new environment
python -m venv decomon-tf-nightly-venv 
. decomon-tf-nightly-venv/bin/activate

# update pip
pip install -U pip

# install tensorflow nightly and matplotlib (dependencies of decomon)
pip install tf-nightly matplotlib

# install decomon without dependencies
# pip install --no-deps decomon  # from pypi
pip install --no-deps -e .  # from source in developer mode, from root directory of decomon repository

# install jupyter to run this notebook!
pip install jupyter
jupyter notebook tutorials/Advanced/tensorboard-and-decomon.ipynb &
```

### On Binder or Colab

**Disclaimer:**
As the environment on binder or colab are using released version of tensorflow (and thus tensorboard), we cannot ensure that the notebook will work as it is for now. (Up to the moment where 2.13.0 will be released)



In [ ]:
import socket

on_colab = "google.colab" in str(get_ipython())  # running on colab?
on_binder = socket.gethostname().startswith("jupyter-")  # running on binder? (not 100% sure but rather robust)

if on_colab or on_binder:
    raise RuntimeError(
        "Do not run this notebook on Binder or Colab. "
        "Please test it locally for now, until tensorflow 2.13.0 is released."
    )

## Imports

In [ ]:
%load_ext tensorboard

from datetime import datetime

import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.models import Sequential

from decomon.models import clone
from decomon.wrapper import get_lower_box, get_upper_box

print("Notebook run using keras:", keras.__version__)

## Initial Keras model

### Build model

The sinusoide funtion is defined on a $[-1 ; 1 ]$ interval. We put a factor in the sinusoide to have several periods of oscillations. 


In [ ]:
x = np.linspace(-1, 1, 1000)
y = np.sin(10 * x)

We approximate this function by a fully connected network composed of 4 hidden layers of size 100, 100, 20 and 20 respectively. Rectified Linear Units (ReLU) are chosen as activation functions for all the neurons. 

In [ ]:
layers = []
layers.append(Dense(100, activation="linear", input_dim=1, name="dense1"))  # specify the dimension of the input space
layers.append(Activation("relu", name="relu1"))
layers.append(Dense(100, activation="linear", name="dense2"))
layers.append(Activation("relu", name="relu2"))
layers.append(Dense(20, activation="linear", name="dense3"))
layers.append(Activation("relu", name="relu3"))
layers.append(Dense(20, activation="linear", name="dense4"))
layers.append(Activation("relu", name="relu4"))
layers.append(Dense(1, activation="linear", name="dense5"))
model = Sequential(layers)

### Fit model

Uncomment the cell below if you want to see the op graph generated during model fit, later in [tensorboard](#Tensorboard).

## Conversion to decomon model

In [ ]:
# convert our model into a decomon model:
decomon_model = clone(model, method="crown")  # method is optionnal

## Visualization

### Serialization to json

In [ ]:
print(model.to_json())

In [ ]:
print(decomon_model.to_json())

### Visualization with Graphviz and pydot

You need to install pydot and graphviz to make it work. If available, uncomment the 2 next cells.

### Tensorboard

We create a log file by setting respectively the Keras and Decomon models to a tensorboard callback.

In [ ]:
logdir = "logs/keras-graph/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=True)
tensorboard_callback.set_model(model)

In [ ]:
logdir = "logs/decomon-graph/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=True)
tensorboard_callback.set_model(decomon_model)

We launch tensorboard to visualize the graph. 
As the op graph is not available without fit, you need to select tag "keras" and graph type "Conceptual graph" on  the right to make it work. 
In "Run" drop-down menu, select "decomon-graph" and then double click on the big node to develop the graph.

In [ ]:
%tensorboard --logdir logs